In [1]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms

import torchvision.datasets as datasets

from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image

In [2]:
data_paths = {
    'train': './data/cassavaleafdata/train',
    'test': './data/cassavaleafdata/test',
    'val': './data/cassavaleafdata/validation'
}

In [5]:
# load image from path
def loader(path):
    return Image.open(path)

img_size = 150

train_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

In [6]:
train_data = datasets.ImageFolder(
    root=data_paths['train'],
    loader=loader,
    transform=train_transforms
)
valid_data = datasets.ImageFolder(
    root=data_paths['val'],
    transform=train_transforms
)
test_data = datasets.ImageFolder(
    root=data_paths['test'],
    transform=train_transforms
)

In [9]:
from torchvision.transforms import ToPILImage

count = 0
for index, (image, label) in enumerate(test_data):
    image = ToPILImage()(image)
    image.save(f'./inference/images/{index}.png')
    count += 1
    if count == 10:
        break

In [12]:
idx2label = {
  0: 'cbb',
  1: 'cbsd',
  2: 'cgm',
  3: 'cmd',
  4: 'healthy',
}

class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=6, kernel_size=5, padding='same'
        )
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(16 * 35 * 35, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc_1(outputs)
        outputs = self.fc_2(outputs)
        outputs = self.fc_3(outputs)
        return outputs

def load_model(model_path, num_classes=5):
    lenet_model = LeNetClassifier(num_classes)
    lenet_model.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('cpu')))
    lenet_model.eval()
    return lenet_model
model = load_model('./model/lenet_model.pt')

def inference(image, model):
  img_size = 150
  img_transform = transforms.Compose([
      transforms.Resize((img_size, img_size)),
      transforms.ToTensor(),
  ])
  img_new = img_transform(image)
  img_new = torch.unsqueeze(img_new, 0)
  with torch.no_grad():
      predictions = model(img_new)
  preds = nn.Softmax(dim=1)(predictions)
  p_max, yhat = torch.max(preds.data, 1)
  return p_max.item()*100, yhat.item()

In [ ]:
for index in range(10):
    image = Image.open(f'./inference/images/{index}.png')
    
    confidence, predicted = inference(image, model)
    
    print(f'Predicted {index}: {idx2label[predicted]} with confidence {confidence:.2f}%, true label: {idx2label[test_data[index][1]]}')

Predicted 0: cmd with confidence 39.78%, true label: cbb
Predicted 1: cbsd with confidence 40.04%, true label: cbb
Predicted 2: cmd with confidence 38.16%, true label: cbb
Predicted 3: cmd with confidence 52.44%, true label: cbb
Predicted 4: cmd with confidence 47.05%, true label: cbb
Predicted 5: cbsd with confidence 45.49%, true label: cbb
Predicted 6: cbsd with confidence 34.01%, true label: cbb
Predicted 7: cmd with confidence 51.32%, true label: cbb
Predicted 8: cmd with confidence 49.87%, true label: cbb
Predicted 9: cmd with confidence 72.97%, true label: cbb
